*Project Walmart*

In [191]:
# The project can be divided into three steps:

# Part 1 : Presentation of the dataset 
# Part 2 : EDA 
# Part 3 : All the necessary preprocessings to prepare data for machine learning
# Part 4 : train a linear regression model (baseline)
# Part 5 : avoid overfitting by training a regularized regression model

In [192]:
import pandas as pd
import numpy as np

import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
import matplotlib.pyplot as plt
import seaborn as sns

In [193]:
!pip install -U kaleido

In [194]:
dataset = pd.read_csv("Walmart_Store_sales.csv")

In [195]:
# To complete this project, your team should:

# Create some visualizations
# Train at least one linear regression model on the dataset, 
# that predicts the amount of weekly sales as a function of the other variables
# Assess the performances of the model by using a metric that is relevant for regression problems
# Interpret the coefficients of the model to identify what features are important for the prediction
# Train at least one model with regularization (Lasso or Ridge) to reduce overfitting

### *Part 1 : Presentation of the dataset* ###

In [196]:
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [197]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

Number of rows : 150
Number of columns : 8



In [198]:
# Basic stats
print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [199]:
# Let's count NaN values in our dataframe
dataset.isna().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

In [200]:
# Let's check the percentage of missing values
print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [201]:
# We decide to drop lines where target values are missing :
dataset = dataset.dropna(subset=['Weekly_Sales'])
print(dataset)

     Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1     13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
3     11.0         NaN    1244390.03           0.0        84.57         NaN   
4      6.0  28-05-2010    1644470.66           0.0        78.89       2.759   
5      4.0  28-05-2010    1857533.70           0.0          NaN       2.756   
..     ...         ...           ...           ...          ...         ...   
145   14.0  18-06-2010    2248645.59           0.0        72.62       2.780   
146    7.0         NaN     716388.81           NaN        20.74       2.778   
147   17.0  11-06-2010     845252.21           0.0        57.14       2.841   
148    8.0  12-08-2011     856796.10           0.0        86.05       3.638   
149   19.0  20-04-2012    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment  
0    214.777523    

In [202]:
# Create usable features from the Date column : The Date column cannot be included as it is in the model.
# Either you can drop this column, or you will create new columns that contain the following numeric features :
# year, month, day, day of week

dataset['Date'] = pd.to_datetime(dataset['Date'], format="%d-%m-%Y")
dataset['Day'] = dataset['Date'].dt.day
dataset['Month'] = dataset['Date'].dt.month
dataset['Year'] = dataset['Date'].dt.year
dataset['Day_of_week'] = dataset['Date'].dt.dayofweek
dataset = dataset.drop('Date', axis=1)
dataset.head()
# 4 = Friday

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day,Month,Year,Day_of_week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,18.0,2.0,2011.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,25.0,3.0,2011.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,28.0,5.0,2010.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,28.0,5.0,2010.0,4.0


In [203]:
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 136
Number of columns : 11

Basics statistics: 


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day,Month,Year,Day_of_week
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000,118.000000,118.000000,118.000000,118.0
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582,16.440678,6.338983,2010.822034,4.0
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428,8.209378,3.173664,0.812628,0.0
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,1.000000,1.000000,2010.000000,4.0
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000,10.000000,4.000000,2010.000000,4.0
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000,16.500000,6.000000,2011.000000,4.0
75%,15.250000,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000,24.000000,9.000000,2011.750000,4.0
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000,31.000000,12.000000,2012.000000,4.0



Percentage of missing values: 


Store            0.000000
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
Day             13.235294
Month           13.235294
Year            13.235294
Day_of_week     13.235294
dtype: float64

### *Part 2 : EDA* ###

In [237]:
fig = px.bar(dataset, x="Year",
             y="Weekly_Sales",
             color='Month',
             title="Bar plot the Weekly_sales for each year", # Month as hue
             height=500,
             width=500,
             )
fig.show()

In [205]:
fig = px.bar(dataset, x="Month",
             y="Weekly_Sales",
             color='Month',
             title="Bar plot the Weekly_sales for each month",
             height=500,
             width=500,
             )
fig.show()

Weekly sales are higher at the end of the years.

### *Part 3 : All the necessary preprocessings to prepare data for machine learning (linear regression) Y = 'Weekly_Sales'* ###

In [206]:
# Let's drop lines containing invalid values or outliers. 
# This concerns the columns : Temperature, Fuel_price, CPI and Unemployment :
Uppertemperature = dataset['Temperature'].mean()+(3*dataset['Temperature'].std())
Lowertemperature = dataset['Temperature'].mean()-(3*dataset['Temperature'].std())
UpperFuelPrice = dataset['Fuel_Price'].mean()+(3*dataset['Fuel_Price'].std())
LowerFuelPrice = dataset['Fuel_Price'].mean()-(3*dataset['Fuel_Price'].std())
UpperCPI = dataset['CPI'].mean()+(3*dataset['CPI'].std())
LowerCPI = dataset['CPI'].mean()-(3*dataset['CPI'].std())
UpperUnemployment = dataset['Unemployment'].mean()+(3*dataset['Unemployment'].std())
LowerUnemployment = dataset['Unemployment'].mean()-(3*dataset['Unemployment'].std())

In [207]:
# Let's create new dataframe without outliers
dataset = dataset[(dataset["Temperature"] < Uppertemperature) & (dataset["Temperature"] > Lowertemperature)]
dataset = dataset[(dataset["Fuel_Price"] < UpperFuelPrice) & (dataset["Fuel_Price"] > LowerFuelPrice)]
dataset = dataset[(dataset["CPI"] < UpperCPI) & (dataset["CPI"] > LowerCPI)]
dataset = dataset[(dataset["Unemployment"] < UpperUnemployment) & (dataset["Unemployment"] > LowerUnemployment)]
dataset

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day,Month,Year,Day_of_week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,18.0,2.0,2011.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,25.0,3.0,2011.0,4.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,28.0,5.0,2010.0,4.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,3.0,6.0,2011.0,4.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,3.0,2.0,2012.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
139,7.0,532739.77,0.0,50.60,3.804,197.588605,8.090,25.0,5.0,2012.0,4.0
143,3.0,396968.80,0.0,78.53,2.705,214.495838,7.343,4.0,6.0,2010.0,4.0
144,3.0,424513.08,0.0,73.44,3.594,226.968844,6.034,19.0,10.0,2012.0,4.0
145,14.0,2248645.59,0.0,72.62,2.780,182.442420,8.899,18.0,6.0,2010.0,4.0


In [208]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = dataset.drop(target_variable, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

X :
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

    Day  Month    Year  Day_of_week  
0  18.0    2.0  2011.0          4.0  
1  25.0    3.0  2011.0          4.0  
4  28.0    5.0  2010.0          4.0  
6   3.0    6.0  2011.0          4.0  
7   3.0    2.0  2012.0          4.0  


In [209]:
# Divide dataset Train set & Test set 
print("Splitting dataset into train set and test set...")
## Then we use train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=0) # We decide to choose the size of the test set at 20%      
                                                                    # 20% of the rows of the data are gonna be in the train and 20% in the test        
print("...Done.")                                                   

Splitting dataset into train set and test set...
...Done.


In [210]:
X = X.astype({"Store":"object","Holiday_Flag":'object', "Year":"float64", "Month":"float64", "Day":"float64"})

In [211]:
numeric_features = X.select_dtypes([np.number]).columns
numeric_features

Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Day', 'Month',
       'Year', 'Day_of_week'],
      dtype='object')

In [212]:
categorical_features = X.select_dtypes("object").columns
categorical_features

Index(['Store', 'Holiday_Flag'], dtype='object')

In [213]:
# Create numerical and categorical variables tranformers with pipelines function
# Use SimpleImputer to change missing numerical values with the median and standardScaler to standardize numerical values
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])# Use SimpleImputer to change missing categorical values with the most frequent value and OneHotEncoder to assign a code to values
# The paramater drop = 'first' delete the first code column to avoid colinearity 
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [214]:
# Create preprocess variable by using ColumnTransformer function to apply the different tranformers in the pipelines
preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
# Apply preprocess on the X_train with fit_transform function (calculate the mean and variance of training set with 'fit' method)
X_train = preprocess.fit_transform(X_train)
# Apply preprocess on the X_test with transform function fit on the training set.
X_test = preprocess.transform(X_test)

### *Part 4 : train a linear regression model (baseline)* ###

In [215]:
# train a linear regression model (baseline)
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

In [216]:
# Make prediction with the function fit on the training set
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)


# Display R2 scores for this model
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000095


In [217]:
scores = cross_val_score(model, X_train, Y_train, cv = 5)
print(f'Les scores obtenus en cross validation sont les suivants : {scores}')
print(f'La moyenne des scores cross_val est de {scores.mean()}')
print(f"L'écart-type des scores cross_val est de {scores.std()}")

Les scores obtenus en cross validation sont les suivants : [0.96696353 0.96508244 0.98023182 0.96465329 0.89361965]
La moyenne des scores cross_val est de 0.9541101472556672
L'écart-type des scores cross_val est de 0.030783765824679964


In [218]:
# -> overfitting

In [219]:
# Let's analyze the values of the model's coefficients to know what features are important for the prediction
# The model's parameters are saved in a .coef_ attribute :
model.coef_

array([-1.14627040e+04, -5.79848294e+04,  7.17469905e+05,  3.24784968e+04,
       -4.95926514e+04,  1.72431945e+04, -6.89502249e+03,  1.90921128e-08,
        2.71338356e+05, -1.25098749e+06,  2.20417284e+06, -1.22743289e+06,
        1.00272537e+05, -6.19830015e+05, -6.56196703e+05, -1.10251725e+06,
        1.79845728e+06,  2.19379595e+05,  2.06681366e+06,  1.01734744e+06,
        5.89257732e+05, -5.77586044e+05,  8.56056660e+05,  9.87464428e+05,
        1.32821080e+06,  5.92403256e+05, -5.35303229e+04])

In [220]:
# Each coefficient can be linked with the name of the corresponding feature by digging into the different pipelines that were used to produce the final version of the X_train/X_test arrays:

column_names = []
for name, pipeline, features_list in preprocess.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Day', 'Month', 'Year', 'Day_of_week', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [221]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-1.146270e+04
Fuel_Price,-5.798483e+04
CPI,7.174699e+05
Unemployment,3.247850e+04
Day,-4.959265e+04
Month,1.724319e+04
Year,-6.895022e+03
Day_of_week,1.909211e-08
x0_2.0,2.713384e+05
x0_3.0,-1.250987e+06


In [222]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Day_of_week,1.909211e-08
Year,6.895022e+03
Temperature,1.146270e+04
Month,1.724319e+04
Unemployment,3.247850e+04
Day,4.959265e+04
x1_1.0,5.353032e+04
Fuel_Price,5.798483e+04
x0_6.0,1.002725e+05
x0_11.0,2.193796e+05


In [223]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.update_yaxes(tickfont_size=9)
fig.show()

In [224]:
# Store x0_4.0 seem to be the most important feature, Store x0_13.0 and Store x0_10.0 are the two following important features

### *Part 5 : avoid overfitting by training 2 regularized regression models (ridge and lasso)* ###

##### *Ridge*

In [225]:
# Cross-validated score for a Ridge model
print("Cross-validation on Ridge...")
ridge_model = Ridge()
scores = cross_val_score(ridge_model, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

Cross-validation on Ridge...
The cross-validated R2-score is :  0.847569652691136
The standard deviation is :  0.06212385843303868


In [226]:
# Grid search on Ridge

# Ridge
# Let's focus on Ridge regularization. We'll train 4 Ridge regressors with different values of the strength 'alpha'
# and analyze the performances as well as the influence on the model's coefficients.
# Perform grid search
print("Grid search...")
ridge_model = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(ridge_model, param_grid = params, cv = 10 ) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.0}
Best R2 score :  0.959543088464633


In [227]:
# alpha : 0.1 is the best hyperparameter

In [228]:
# Performance assessment
# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000094


In [229]:
0.9390452740506877-0.9352216314000095

0.0038236426506782495

##### *Lasso*

In [230]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
# Cross-validated score for a Lasso model
print("Cross-validation on Ridge...")
lasso_model = Lasso()
scores = cross_val_score(lasso_model, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

Cross-validation on Ridge...
The cross-validated R2-score is :  0.961355309012335
The standard deviation is :  0.02279200457963867


e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.921e+11, tolerance: 2.754e+09

e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.496e+11, tolerance: 2.737e+09

e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.719e+11, tolerance: 2.680e+09

e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Object

In [231]:
# Grid search on Lasso
# Let's focus on Lasso regularization. We'll train 4 Lasso regressors with different values of the strength 'alpha'
# and analyze the performances as well as the influence on the model's coefficients.
# Perform grid search
print("Grid search...")
lasso_model = Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(lasso_model, param_grid = params, cv = 10 ) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...


e:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:680: UserWarning:

With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator

e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning:

Coordinate descent with no regularization may lead to unexpected results and is discouraged.

e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.927e+11, tolerance: 2.754e+09 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.

e:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:680: UserWarning:

With alpha=0, this algorithm does not converge

...Done.
Best hyperparameters :  {'alpha': 1.0}
Best R2 score :  0.961355309012335


e:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.033e+11, tolerance: 3.010e+09



In [232]:
# Performance assessment
# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.9864577366096275
R2 score on test set :  0.9390452740506877


In [233]:
 0.961355309012335-0.9352216314000095

0.026133677612325545

### *Conclusion* ###

-  Ridge and Lasso don't fight overfitting enough, there is no difference between these models
-  The most important feature is the target itself